In [1]:
import os
import pandas as pd

from pathlib import Path
from glob import glob
from datetime import datetime

!pip install xlrd

In [2]:
current_path = Path(os.getcwd())
root = current_path.parent.absolute()
data_folder = str(root) + '/raw data/'

In [3]:
types = ["xlsx", "csv", "xls"]

os.chdir(data_folder)

tanguy = {}

print("-- XLSX FILES --")
for file_path in glob("*.xlsx"):
    file_name = file_path.split(".")[0]
    if file_name == 'giga_base':break

    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        tanguy[sheet] = file.parse(sheet)
    print()
    
    
print("\n-- XLS FILES --") 
for file_path in glob("*.xls"):
    file_name = file_path.split(".")[0]

    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        tanguy[sheet] = file.parse(sheet)
    print()
    
print("\n-- csv FILES --")
for file_path in glob("*.csv"):
    file_name = file_path.split(".")[0]
    print(file_name)
    file_df = pd.read_csv(file_path, sep=';')
    tanguy[file_name] = file_df

print("\n-- CSV FILES --")
for file_path in glob("*.CSV"):
    file_name = file_path.split(".")[0]
    print(file_name)
    file_df = pd.read_csv(file_path, sep=';')
    tanguy[file_name] = file_df

-- XLSX FILES --
Betting Against Beta Equity Factors Daily sheets:
-->BAB Factors
-->HML Devil

bloom_data_2 sheets:
-->CL1
-->CL2
-->CL3
-->Euribor3
-->Bund10
-->FUTURES MATURITIES

Quality Minus Junk Factors Daily sheets:
-->QMJ Factors
-->HML Devil

Value and Momentum Everywhere Factors Monthly sheets:
-->VME Factors

Volatility_data_treated sheets:
-->Sheet1


-- XLS FILES --
sentiment sheets:
-->SENTIMENT


-- csv FILES --
F-F_Momentum_Factor_daily
F-F_Research_Data_5_Factors_2x3_daily
F-F_ST_Reversal_Factor_daily

-- CSV FILES --
F-F_Momentum_Factor_daily
F-F_Research_Data_5_Factors_2x3_daily
F-F_ST_Reversal_Factor_daily


In [4]:

# Intersect avec dates VIX
# Donnés monthly à intersect
# supprimer value and momentum

keys = list(tanguy.keys())
for key in keys:
    if key !='FUTURES MATURITIES':
        if type(tanguy[key].index)!=pd.DatetimeIndex:
            date_col = [col for col in ['Date','DATE','Unnamed: 0','Reported data', 'time'] if col in tanguy[key].columns]
            assert len(date_col)==1, "Date column is not unique"+ str( key)
            if key in ['F-F_ST_Reversal_Factor_daily', 'F-F_Research_Data_5_Factors_2x3_daily', 'F-F_Momentum_Factor_daily']:
                tanguy[key][date_col]= tanguy[key][date_col].applymap(lambda x: datetime.strptime(x,'%d/%m/%Y'))
            tanguy[key].set_index(date_col, inplace=True)
            if key == 'VME Factors':
                # Resampling monthly -> daily
                #The value for month n is the one released on the last day of the month n-1, as it is one agents use during month n.
                start_date = tanguy["VME Factors"].index.min() - pd.DateOffset(day=1)
                end_date = tanguy["VME Factors"].index.max() + pd.DateOffset(day=31)
                dates = pd.date_range(start_date, end_date, freq='D')
                tanguy["VME Factors"] = tanguy["VME Factors"].reindex(dates, method='ffill')
        tanguy[key].index.names = ["Date"]
        #Adding a suffix
        tanguy[key].rename(columns = lambda x: x+'_'+key,inplace=True)


In [5]:
#Now merging everything into big_base

big_base = tanguy[keys[0]]['2000':].join(tanguy[keys[1]]['2000':], how='outer')


for key in keys[2:]:
    print(key)
    if key not in ['FUTURES MATURITIES']: big_base = big_base.join(tanguy[key]['2000':], how='outer')

big_base.interpolate(method='time' ,axis='index', limit_area='inside', inplace=True)


CL1
CL2
CL3
Euribor3
Bund10
FUTURES MATURITIES
QMJ Factors
VME Factors
Sheet1
SENTIMENT
F-F_Momentum_Factor_daily
F-F_Research_Data_5_Factors_2x3_daily
F-F_ST_Reversal_Factor_daily


In [10]:
os.chdir(root)
big_base.to_excel(str(root) + 'giga_base.xlsx', na_rep='#N/A N/A')
